In [1]:
# (G,M,r) -> v    in m,kg,s
# [3,-1,-2]  [0,1,0]  [1,0,0] -> [1,0,-1]


In [1]:
import torch
import numpy as np
units_in = torch.tensor([[3,-1,-2],[0,1,0],[1,0,0]]).T.float()
units_out = torch.tensor([[1,0,-1]]).T.float()

In [2]:
units_out.shape

torch.Size([3, 1])

In [3]:
G = 6

In [4]:
N=1000
Mr =  torch.exp(torch.rand(N,2)*np.log(100)+np.log(.1))
Y = v = (G*Mr[:,0]/Mr[:,1]).sqrt()
X = torch.cat([G*torch.ones(N,1),Mr],dim=-1)

In [5]:
X

tensor([[6.0000, 3.2928, 1.2178],
        [6.0000, 4.7421, 0.4511],
        [6.0000, 0.2424, 0.7075],
        ...,
        [6.0000, 0.1964, 3.7555],
        [6.0000, 4.4670, 3.7429],
        [6.0000, 0.4674, 4.3631]])

In [6]:
#Y

In [7]:
from unitless.layers import DimensionalMLP,StandardMLP

In [8]:
NN = DimensionalMLP(units_in,units_out)

In [9]:
#NN(X)

In [10]:
  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import os.path
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

class physics(Dataset):
    def __init__(self,root=None,train=True):
        super().__init__()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X.numpy(), Y.numpy(), train_size=100)
        self.X = torch.from_numpy(self.X_train if train else self.X_test).float()
        self.Y = torch.from_numpy(self.y_train if train else self.y_test).float()
        self.dim = self.X.shape[1]
    def __getitem__(self,idx):
        return self.X[idx],self.Y[idx]
    def __len__(self):
        return self.X.shape[0]

In [11]:

import torch
from torch.utils.data import DataLoader
from torch.optim import SGD,Adam,AdamW
from oil.utils.utils import LoaderTo, cosLr, islice, export
from oil.tuning.study import train_trial
from oil.datasetup.datasets import CIFAR10, split_dataset
from oil.architectures.img_classifiers import layer13
from oil.utils.parallel import try_multigpu_parallelize
from oil.tuning.args import argupdated_config
from oil.model_trainers.classifier import Regressor
from functools import partial

def makeTrainer(*,dataset=physics,network=DimensionalMLP,num_epochs=100,
                bs=50,lr=1e-3,optim=AdamW,device='cuda',trainer=Regressor,
                split={'train':1},opt_config={},
                trainer_config={'log_dir':None,'log_args':{'minPeriod':0}},save=False):

    # Prep the datasets splits, model, and dataloaders
    datasets = split_dataset(dataset(f'~/datasets/{dataset}/'),splits=split)
    datasets['test'] = dataset(f'~/datasets/{dataset}/', train=False)

    device = torch.device(device)
    model = network(units_in,units_out).to(device)
    dataloaders = {k:LoaderTo(DataLoader(v,batch_size=bs,shuffle=(k=='train'),
                num_workers=0,pin_memory=False),device) for k,v in datasets.items()}
    dataloaders['Train'] = dataloaders['train']
    # Add some extra defaults if SGD is chosen
    if optim==SGD: opt_config={**{'momentum':.9,'nesterov':True},**opt_config}
    opt_constr = partial(optim, lr=lr, **opt_config,weight_decay=1e-4)
    lr_sched = cosLr(num_epochs)
    return trainer(model,dataloaders,opt_constr,lr_sched,**trainer_config)



In [12]:
# network = StandardMLP
# network = DimensionalMLP
#T = makeTrainer(network=StandardMLP)

In [13]:
#T.train(1000)

In [16]:
# import matplotlib.pyplot as plt
# T.logger.scalar_frame.plot()
# plt.yscale('log')

In [17]:
T2 = makeTrainer(network=DimensionalMLP,lr=1e-2)
T2.train(1000)
T2.logger.scalar_frame.plot()
plt.yscale('log')

   Minibatch_Loss  Train_MSE   lr0  test_MSE
0             NaN        NaN  0.01       NaN
    Minibatch_Loss  Train_MSE       lr0  test_MSE
74             NaN        NaN  0.007058       NaN
     Minibatch_Loss  Train_MSE      lr0  test_MSE
150             NaN        NaN  0.00152       NaN
     Minibatch_Loss  Train_MSE       lr0  test_MSE
228             NaN        NaN  0.009557       NaN
     Minibatch_Loss  Train_MSE       lr0  test_MSE
305             NaN        NaN  0.004686       NaN
     Minibatch_Loss  Train_MSE       lr0  test_MSE
383             NaN        NaN  0.000199       NaN
     Minibatch_Loss  Train_MSE       lr0  test_MSE
460             NaN        NaN  0.008002       NaN
     Minibatch_Loss  Train_MSE       lr0  test_MSE
538             NaN        NaN  0.002255       NaN


KeyboardInterrupt: 

In [ ]:
T2.model(X.cuda())

In [22]:
x = (T2.model.units_in_vec.cuda(),X.cuda())


In [23]:
T2.model.network(x)

(tensor([[ 2.0870],
         [ 0.7275],
         [-0.0335]], device='cuda:0', grad_fn=<MmBackward>),
 tensor([[ 3.8870e-01],
         [ 2.2282e+00],
         [ 5.4234e+00],
         [ 4.7517e+00],
         [ 8.8150e+00],
         [ 1.3782e+00],
         [ 4.7404e+00],
         [ 2.0920e+00],
         [ 2.1842e+00],
         [ 7.9708e+00],
         [ 2.9198e+00],
         [ 1.1810e+00],
         [ 3.8636e+00],
         [ 3.3909e+00],
         [ 1.6136e+00],
         [ 1.4338e+00],
         [ 1.9930e+00],
         [ 5.7638e-01],
         [ 1.6888e+01],
         [ 5.2715e+00],
         [ 3.0450e+00],
         [ 3.3371e+00],
         [ 4.1121e+00],
         [ 6.5841e+00],
         [ 8.2477e-01],
         [ 4.2466e+00],
         [ 8.5763e+00],
         [ 8.1437e-01],
         [ 4.3148e+00],
         [ 4.9359e+00],
         [ 1.0539e+00],
         [ 4.6623e+00],
         [ 4.1706e-01],
         [ 3.6815e+00],
         [ 3.8131e-01],
         [ 2.9403e+00],
         [ 1.7074e+00],
         [ 

In [64]:
'∂'

'∂'

In [65]:
∂

SyntaxError: invalid character in identifier (<ipython-input-65-50ebee7ad286>, line 1)